In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from dataset import myData

原数据集训练样本数：19634，去除空值后样本数：10452
原数据集测试样本数：2203，去除空值后样本数：1169


In [5]:
data=myData()

In [6]:
X_train,y_train,X_test,y_test=data.dataset01()

X_train'shape:(10452, 1000, 12)  y_train'shape:(10452, 2)  X_test'shape:(1169, 1000, 12)  y_test'shape:(1169, 2)


In [4]:
from tensorflow import keras
from tensorflow.keras import layers

In [25]:

def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    
    x =layers.Dense(12)(x)
   
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


In [26]:

def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)


In [27]:
input_shape = X_train.shape[1:]
n_classes = len(np.unique(y_train))
model = build_model(
    input_shape,
    head_size=128,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["accuracy"],
)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 1000, 12)]   0           []                               
                                                                                                  
 layer_normalization_26 (LayerN  (None, 1000, 12)    24          ['input_6[0][0]']                
 ormalization)                                                                                    
                                                                                                  
 conv1d_32 (Conv1D)             (None, 1000, 4)      52          ['layer_normalization_26[0][0]'] 
                                                                                                  
 dropout_33 (Dropout)           (None, 1000, 4)      0           ['conv1d_32[0][0]']        

 tf.__operators__.add_30 (TFOpL  (None, 1000, 12)    0           ['dropout_40[0][0]',             
 ambda)                                                           'tf.__operators__.add_29[0][0]']
                                                                                                  
 layer_normalization_31 (LayerN  (None, 1000, 12)    24          ['tf.__operators__.add_30[0][0]']
 ormalization)                                                                                    
                                                                                                  
 conv1d_42 (Conv1D)             (None, 1000, 4)      52          ['layer_normalization_31[0][0]'] 
                                                                                                  
 dropout_41 (Dropout)           (None, 1000, 4)      0           ['conv1d_42[0][0]']              
                                                                                                  
 conv1d_43

In [28]:
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    epochs=200,
    batch_size=64,
    callbacks=callbacks,
)

Epoch 1/200
  4/131 [..............................] - ETA: 2:31:59 - loss: 0.7209 - accuracy: 0.5195

KeyboardInterrupt: 

In [7]:
model.evaluate(X_test, y_test, verbose=1)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1000, 12)]   0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 1000, 12)    24          ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 1000, 12)    26124       ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                              

 mbda)                                                            'tf.__operators__.add_5[0][0]'] 
                                                                                                  
 layer_normalization_7 (LayerNo  (None, 1000, 12)    24          ['tf.__operators__.add_6[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv1d_6 (Conv1D)              (None, 1000, 4)      52          ['layer_normalization_7[0][0]']  
                                                                                                  
 dropout_7 (Dropout)            (None, 1000, 4)      0           ['conv1d_6[0][0]']               
                                                                                                  
 conv1d_7 (Conv1D)              (None, 1000, 12)     60          ['dropout_7[0][0]']              
          

KeyboardInterrupt: 